In [125]:
import numpy as np

I would like my boards to be generated via a numpy array- having L-systems be used by casting a rule to each cell of the board, and recursively applying said rule.

In [126]:
def init_board(size:int=5):
    return np.zeros(shape=(size,size), dtype=int)

init_board(5)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [143]:
# unlike jumanji, each agent here is assigned one number
# and use that number throughout the duration of the generation
# a mapping is then done to create suitable jumanji boards
# agents are thus for the following named by their number.

def initialise_starting_points(board:np.ndarray, agent_count:int=3):
    l = len(board) # board is assumed to be square
    for agent in range(1, 1+agent_count):
        tries = 0
        while(1):
            tries += 1
            if tries == 100:
                print(f'ERROR - NO BOARD SPACES FOUND AFTER {100} TRIES')
                break

            idx0 = np.random.randint(0, l)
            idx1 = np.random.randint(0, l)
            if board[idx0, idx1] != 0.0:
                continue
            else:
                board[idx0, idx1] = agent
                break
    return board


board_size = 5
agent_count = 3

board = init_board(board_size)
board = initialise_starting_points(board, 3)
print(board)


def assign_agent_states(board, agent_count):
    # each agent is going to be assigned a self-state 
    # as the following tuple: (number, head_pos, tail_pos)
    agents = []
    for agent in range(1, 1+agent_count):
        pos = np.argwhere(board==agent)[0]
        agents.append([agent, pos, pos])
    return agents


agents = assign_agent_states(board, agent_count)
print(agents)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 3 0 0]
 [0 1 0 0 0]
 [2 0 0 0 0]]
[[1, array([3, 1]), array([3, 1])], [2, array([4, 0]), array([4, 0])], [3, array([2, 2]), array([2, 2])]]


Q: what rules should be created to L-systemize this board? Substitution rules!!

1) push-out in a direction from one of the ends
this should happen most of the time
2) pull-in in a direction from one of the ends
this should happen less-so
3) How long should (1) and (2) occur for? What are good ratios of push and pull?
4) Should (1) and (2) be coupled across agents? Should it just be per-agent?

In [144]:
def find_next_pos(board, loc):
    empty_places = np.argwhere(board==0)
    empty_place_distances = np.sqrt(np.sum((loc-empty_places)**2, axis=1))
    empty_neighbour_places = np.argwhere(empty_place_distances<1.111)
    empty_neighbour_coords = np.take(empty_places, empty_neighbour_places, axis=0)
    return empty_neighbour_coords[:,0,:]
    
print(board)
print(agents[0][1])
nb_coords = find_next_pos(board, agents[0][1])
nb_coords

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 3 0 0]
 [0 1 0 0 0]
 [2 0 0 0 0]]
[3 1]


array([[2, 1],
       [3, 0],
       [3, 2],
       [4, 1]])

In [145]:
def push(board, agent):
    growth_choice = np.random.randint(1, 3)
    nbs = find_next_pos(board, agent[growth_choice])
    try:
        growth_direction = np.random.randint(0, len(nbs))
    except:
        return board, agent
    board[tuple(nbs[growth_direction])] = agent[0]
    agent[growth_choice] = nbs[growth_direction]
    return board, agent

print(board, agents[1])
push(board, agents[1])

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 3 0 0]
 [0 1 0 0 0]
 [2 0 0 0 0]] [2, array([4, 0]), array([4, 0])]


(array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 3, 0, 0],
        [0, 1, 0, 0, 0],
        [2, 2, 0, 0, 0]]),
 [2, array([4, 0]), array([4, 1])])

In [146]:
board = initialise_starting_points(init_board(5), 3)
agents = assign_agent_states(board, 3)

def show_pushes(board, agents):
    print(board, '\n')
    agent_choice = 1
    agent = agents[agent_choice-1]
    for i in range(10):
        board, agent = push(board, agent)
        print(board, '\n')

show_pushes(board, agents)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 3]
 [0 0 2 1 0]
 [0 0 0 0 0]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 3]
 [0 0 2 1 1]
 [0 0 0 0 0]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 3]
 [0 0 2 1 1]
 [0 0 0 0 0]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [0 0 0 0 0]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [0 0 0 0 1]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [0 0 0 1 1]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [0 0 1 1 1]] 

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [0 1 1 1 1]] 

[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [0 1 1 1 1]] 

[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [1 1 1 1 1]] 

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 1 3]
 [0 0 2 1 1]
 [1 1 1 1 1]] 

